In [ ]:
# not run in GCP
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/MyDrive/0-ECBM_4040-DL/DL_final_project')

## Load Packages

In [ ]:
# !pip install imgaug
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from utils import download_data, load_data, data_augmentation, get_labels
from visualization import show_sample, plot_loss, show_prediction
from models import init_model_1, init_model_2, training, save_model, predict

In [ ]:
data_path = './data'
checkpoint_filepath = './modelcheckpoints'
model_path = './model'

In [ ]:
# not run in GCP
# data_path = '/content/drive/MyDrive/0-ECBM_4040-DL/DL_final_project/data'
# checkpoint_filepath = '/content/drive/MyDrive/0-ECBM_4040-DL/DL_final_project/modelcheckpoints'
# model_path = '/content/drive/MyDrive/0-ECBM_4040-DL/DL_final_project/model'

## Load data

In [ ]:
# if data is not downloaded, will download first
train_generator, val_generator, test_generator = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=None, seed=None, load_test=True)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
Found 10000 validated image filenames belonging to 200 classes.
Testing data shape: (10000, 64, 64, 3)
End loading!


## Data augmentation

In [ ]:
# parameters for data_augmentation function
# use ?data_aug_args to see details for those paramters
data_aug_args = dict(CoarseDropout_range=(0.0, 0.05),
                     CoarseDropout_size_percent=(0.02, 0.25),
                     Affine_translate_percent=(-0.2, 0.2),
                     Affine_scale=(0.5, 1.5),
                     Affine_shear=(-20, 20),
                     Affine_rotate=(-45, 45),
                     Flip_percent=0.5,
                     GaussianBlur_sigma=(0.0, 3.0),
                     CropAndPad_percent=(-0.25, 0.25),
                     Multiply=(0.5, 1.5),
                     LinearContrast=(0.4, 1.6),
                     AdditiveGaussianNoise_scale=0.2*255)
# get augmnetation function
# complicated=True: augmentation for network 2, false: augmentation for network 1
aug = data_augmentation(complicated=False, **data_aug_args)

# get augmented data generator
train_generator_aug, val_generator_aug = load_data(data_path, img_width=64, img_height=64, 
                                                   batch_size=128, augmentation=aug, 
                                                   seed=None)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!


## Model 1 Initialization

In [ ]:
# init model 1 with the same architecture in the paper
model1 = init_model_1((None,None,3))
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# compile model
# optimizer: Adam
opt = Adam()
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### Train model 1 with 32x32 data and 8 epochs

In [ ]:
# get data generator
train_generator_32, val_generator_32 = load_data(data_path, img_width=32, img_height=32, 
                                           batch_size=128, augmentation=None, seed=None)
# train model
callback = ReduceLROnPlateau(patience=5, min_lr=6e-7)
model1_1, history1_1 = training(model1, 'model1_1', train_generator_32, val_generator_32, 
                           callback, checkpoint_filepath, epochs=8)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 32, 32, 3)
Validation data shape: (20000, 32, 32, 3)
End loading!
Epoch 1/8
200/200 [==============================] - 173s 864ms/step - loss: 4.7436 - accuracy: 0.0637 - val_loss: 5.6997 - val_accuracy: 0.0097 - lr: 0.0010
Epoch 2/8
200/200 [==============================] - 171s 855ms/step - loss: 4.3448 - accuracy: 0.1113 - val_loss: 5.2052 - val_accuracy: 0.0286 - lr: 0.0010
Epoch 3/8
200/200 [==============================] - 171s 855ms/step - loss: 4.1007 - accuracy: 0.1464 - val_loss: 4.4308 - val_accuracy: 0.0898 - lr: 0.0010
Epoch 4/8
200/200 [==============================] - 171s 856ms/step - loss: 3.9054 - accuracy: 0.1732 - val_loss: 4.1942 - val_accuracy: 0.1256 - lr: 0.0010
Epoch 5/8
200/200 [==============================] - 171s 855ms/step - loss: 3.7320 - accuracy: 0.2036 - val_loss: 3.9762 - val_accuracy: 0.1548 - lr: 0.0010
Epoch 6/

In [ ]:
# save model periodically
save_model(model1_1, 'model1_1', model_path)

model1_1_12-19_21-03.h5 saved at ./model!


### Train model 1 with 64x64 data and 15 epochs

In [ ]:
# get data generator
train_generator_64, val_generator_64 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=None, seed=None)
# train model
model1_2, history1_2 = training(model1_1, 'model1_2', train_generator_64, val_generator_64, 
                           callback, checkpoint_filepath, epochs=15)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/15
200/200 [==============================] - 657s 3s/step - loss: 3.4248 - accuracy: 0.2752 - val_loss: 3.8398 - val_accuracy: 0.1708 - lr: 0.0010
Epoch 2/15
200/200 [==============================] - 653s 3s/step - loss: 3.2477 - accuracy: 0.2971 - val_loss: 3.4940 - val_accuracy: 0.2228 - lr: 0.0010
Epoch 3/15
200/200 [==============================] - 653s 3s/step - loss: 3.1201 - accuracy: 0.3204 - val_loss: 3.5342 - val_accuracy: 0.2211 - lr: 0.0010
Epoch 4/15
200/200 [==============================] - 653s 3s/step - loss: 3.0076 - accuracy: 0.3362 - val_loss: 3.4691 - val_accuracy: 0.2409 - lr: 0.0010
Epoch 5/15
200/200 [==============================] - 653s 3s/step - loss: 2.9203 - accuracy: 0.3473 - val_loss: 3.4291 - val_accuracy: 0.2390 - lr: 0.0010
Epoch 6/15
200/200

In [ ]:
# save model periodically
save_model(model1_2, 'model1_2', model_path)

model1_2_12-19_23-47.h5 saved at ./model!


### Train model 1 with 16x16 data and 5 epochs

In [ ]:
# get data generator
train_generator_16, val_generator_16 = load_data(data_path, img_width=16, img_height=16, 
                                           batch_size=128, augmentation=None, seed=None)
# train model
model1_3, history1_3 = training(model1_2, 'model1_3', train_generator_16, val_generator_16, 
                           callback, checkpoint_filepath, epochs=5)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 16, 16, 3)
Validation data shape: (20000, 16, 16, 3)
End loading!
Epoch 1/5
200/200 [==============================] - 56s 280ms/step - loss: 3.6970 - accuracy: 0.1988 - val_loss: 3.6112 - val_accuracy: 0.2152 - lr: 0.0010
Epoch 2/5
200/200 [==============================] - 55s 277ms/step - loss: 3.1337 - accuracy: 0.2840 - val_loss: 3.5588 - val_accuracy: 0.2263 - lr: 0.0010
Epoch 3/5
200/200 [==============================] - 55s 277ms/step - loss: 2.9498 - accuracy: 0.3224 - val_loss: 3.4739 - val_accuracy: 0.2382 - lr: 0.0010
Epoch 4/5
200/200 [==============================] - 55s 277ms/step - loss: 2.7399 - accuracy: 0.3602 - val_loss: 3.4253 - val_accuracy: 0.2551 - lr: 0.0010
Epoch 5/5
200/200 [==============================] - 55s 277ms/step - loss: 2.5912 - accuracy: 0.3966 - val_loss: 3.3134 - val_accuracy: 0.2623 - lr: 0.0010


In [ ]:
# save model periodically
save_model(model1_3, 'model1_3', model_path)

model1_3_12-19_23-57.h5 saved at ./model!


### Train model 1 with 64x64 data and 15 epochs

In [ ]:
# get data generator
train_generator_64, val_generator_64 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=None, seed=None)
# train model
model1_4, history1_4 = training(model1_3, 'model1_4', train_generator_64, val_generator_64, 
                           callback, checkpoint_filepath, epochs=15)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/15
200/200 [==============================] - 653s 3s/step - loss: 2.4367 - accuracy: 0.4443 - val_loss: 2.7572 - val_accuracy: 0.3751 - lr: 0.0010
Epoch 2/15
200/200 [==============================] - 653s 3s/step - loss: 2.0930 - accuracy: 0.5083 - val_loss: 2.7404 - val_accuracy: 0.3756 - lr: 0.0010
Epoch 3/15
200/200 [==============================] - 652s 3s/step - loss: 1.9781 - accuracy: 0.5350 - val_loss: 2.6057 - val_accuracy: 0.3925 - lr: 0.0010
Epoch 4/15
200/200 [==============================] - 653s 3s/step - loss: 1.8943 - accuracy: 0.5509 - val_loss: 2.4974 - val_accuracy: 0.4119 - lr: 0.0010
Epoch 5/15
200/200 [==============================] - 652s 3s/step - loss: 1.8356 - accuracy: 0.5660 - val_loss: 2.4524 - val_accuracy: 0.4211 - lr: 0.0010
Epoch 6/15
200/200

In [ ]:
# save model periodically
save_model(model1_4, 'model1_4', model_path)

model1_4_12-20_02-41.h5 saved at ./model!


### Train model 1 with 64x64 augmented data and 75 epochs

In [ ]:
callback = ReduceLROnPlateau(patience=5, min_lr=6e-7)
model1_4 = tf.keras.models.load_model(model_path+'/model1_4_12-20_02-41.h5')

In [ ]:
# get augmnetation function
# complicated=True: augmentation for network 2, false: augmentation for network 1
aug = data_augmentation(complicated=False, **data_aug_args)
# get data generator
train_generator_aug, val_generator_aug = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=aug, seed=None)
# train model
model1_5, history1_5 = training(model1_4, 'model1_5', train_generator_aug, val_generator_aug, 
                           callback, checkpoint_filepath, epochs=75)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/75
200/200 [==============================] - 658s 3s/step - loss: 3.2838 - accuracy: 0.2901 - val_loss: 3.9038 - val_accuracy: 0.2050 - lr: 0.0010
Epoch 2/75
200/200 [==============================] - 653s 3s/step - loss: 3.0117 - accuracy: 0.3256 - val_loss: 4.1169 - val_accuracy: 0.1884 - lr: 0.0010
Epoch 3/75
200/200 [==============================] - 653s 3s/step - loss: 2.9457 - accuracy: 0.3389 - val_loss: 3.7188 - val_accuracy: 0.2189 - lr: 0.0010
Epoch 4/75
200/200 [==============================] - 653s 3s/step - loss: 2.8884 - accuracy: 0.3482 - val_loss: 3.4911 - val_accuracy: 0.2490 - lr: 0.0010
Epoch 5/75
200/200 [==============================] - 653s 3s/step - loss: 2.8286 - accuracy: 0.3630 - val_loss: 3.7666 - val_accuracy: 0.2224 - lr: 0.0010
Epoch 6/75
200/200

In [ ]:
# save model periodically
save_model(model1_5, 'model1_5', model_path)

Due to the limitation of computing resource, we could not run all 75 epochs in 12 hours and the training ended in the 66th epoch.